In [1]:
#default_exp core

## How do you evaluate monolingual embeddings?

What constitutes a good embedding? It would be easy to provide a cursory answer - a good embedding is one that lends itself well to a downstream task. While 100% true and accurate, this answer does not allow us to speak to the goodness of embeddings directly, without having to train an additional model.

Another way to look at this would be to define that embeddings are valuable and useful to the extent that they encode information about the language and the world around us. This is in line with the reasoning behind why embeddings were created in the first place - we want to train our embeddings, or our language model, on vast amount of unlabeled text, in a way that encodes syntactic and semantic information that can give us a boost on a downstream task where we have labels but the dataset might be of limited size.

Taking the second definition, we can attempt to query our embeddings on textual examples and evaluate the accuracy of the anwsers. In its simplest form, we can perform algebraic operations in the embedding space ("king" - "man" + "woman" = ?) and use this as a mechanism for evaluation. While not without [issues](https://www.aclweb.org/anthology/W16-2503.pdf), this approach does allow us to say something about the structure of the trained embedding space.

To demonstrate this approach, let's use the embeddings from a classic, seminal paper [Efficient Estimation of Word Representations in Vector Space paper](https://arxiv.org/abs/1301.3781) by Tomas Mikolov et al.

We can download the embeddings using fastai (they were originally shared by the authors [here](https://code.google.com/archive/p/word2vec/)).

Please note - the file is 1.7 GB compressed.

In [2]:
from fastai.data.all import untar_data
embedding_path = untar_data('https://storage.googleapis.com/text-embeddings/GoogleNews-vectors-negative300.bin.tar.gz')

Now let's load the embeddings using `gensim`.

In [3]:
from gensim.models.keyedvectors import KeyedVectors

gensim_embeddings = KeyedVectors.load_word2vec_format(embedding_path, binary=True)

In [4]:
len(gensim_embeddings.index2entity), gensim_embeddings['cat'].shape

(3000000, (300,))

3 million distinct embeddings, each of dimensionality 300!

Let's perform the evaluation using the original list of `question-words.txt` as used in the paper (and that was shared by the authors on github [here](https://github.com/tmikolov/word2vec/blob/master/questions-words.txt)).

We could use the functionality built into `gensim` to run the evaluation, but this might make it tricky to evaluate embeddings that we train ourselves, or should we want to modify the list of queries.

Instead, let's perform the evaluation using code that we develop in this repository. As a starting point, all we need is an array of embeddings and a list with words corresponding to each vector!

<!-- We will use [annoy](https://github.com/spotify/annoy) for approximate nearest neighbor lookup. Upon the first run, the embeddings will be added to an index and multiple trees enabling the search will be constructed. Given the size of these embeddings, this took around 5 minutes for me.  -->

In [5]:
#export
import numpy as np

class Embeddings():
    def __init__(self, embeddings, index2word):
        '''embeddings - numpy array of embeddings, index2word - list of words corresponding to embeddings'''
        assert len(embeddings) == len(index2word)
        self.vectors = embeddings
        self.i2w = index2word
        self.w2i = {w:i for i, w in enumerate(index2word)}
            
    def analogy(self, a, b, c, n=5, discard_question_words=True):
        '''
        a is to b as c is to ?
        
        Performs the following algebraic calculation: result = emb_a - emb_b + emb_c
        Looks up n closest words to result.
        
        Implements the embedding space math behind the famous word2vec example:
        king - man + woman = queen
        '''
        question_word_indices = [self.w2i[word] for word in [a, b, c]]
        a, b, c = [self.vectors[idx] for idx in question_word_indices] 
        result = a - b + c
        
        if discard_question_words: return self.nn_words_to(result, question_word_indices, n)
        else:                      return self.nn_words_to(result, n=n)
        
    def nn_words_to(self, vector, skip_indices=[], n=5):
        nn_indices = self.word_idxs_ranked_by_cosine_similarity_to(vector)
        nn_words = []
        for idx in nn_indices:
            if idx in skip_indices: continue
            nn_words.append(self.i2w[idx])
            if len(nn_words) == n: break
        
        return nn_words
    
    def word_idxs_ranked_by_cosine_similarity_to(self, vector):
        return np.flip(
            np.argsort(self.vectors @ vector / (self.vectors_lengths() * np.linalg.norm(vector, axis=-1)))
        )
    
    def vectors_lengths(self):
        if not hasattr(self, 'vectors_length_cache'):
            self.vectors_length_cache = np.linalg.norm(self.vectors, axis=-1)
        return self.vectors_length_cache
    
    def __getitem__(self, word):
        return self.vectors[self.w2i[word]]
    
    @classmethod
    def from_txt_file(cls, path_to_txt_file, limit=None):
        '''create embeddings from word2vec embeddings text file'''
        index, vectors = [], []
        with open(path_to_txt_file) as f:
            f.readline() # discarding the header line
            for line in f:
                try:
                    embedding = np.array([float(s) for s in line.split()[1:]])
                    if embedding.shape[0] != 300: continue
                    vectors.append(embedding)
                    index.append(line.split()[0])
                except ValueError: pass # we may have encountered a 2 word embedding, for instance 'New York' or 'w dolinie'
                if limit is not None and len(vectors) == limit: break
        return cls(np.stack(vectors), index)

In [6]:
gensim_embeddings.vectors[:30000].shape

(30000, 300)

In [7]:
# grabbing just the vectors and mapping of vectors to words from gensim embeddings and instatiating our own embedding object
# let's stick to just 50_000 of the most popular words so that the computation will run faster

embeddings = Embeddings(gensim_embeddings.vectors[:50_000], gensim_embeddings.index2word[:50_000])

Now that we have the Embeddings in place, we can run some examples. France is to Paris as ? is to Warsaw...

In [8]:
%%time
embeddings.analogy('France', 'Paris', 'Warsaw', 5)

CPU times: user 108 ms, sys: 24 ms, total: 132 ms
Wall time: 33.3 ms


['Poland', 'Polish', 'Romania', 'Lithuania', 'Poles']

Got that one right! Now let's try the classic example of king - man + women = ?

In [9]:
%%time
embeddings.analogy('king', 'man', 'woman', 5)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 7.17 ms


['queen', 'monarch', 'princess', 'prince', 'kings']

We get it right as well!

Despite kings and queens not being discussed that often in the news today, this is still a great and slightly unexpected performance. Why should such an algebraic structure emerge when trained on a lot of text data in the first place? But yet it does!

Let's explore the performance further, by running through the list of question-answer pairs.

In [10]:
#export

from collections import defaultdict
import pandas as pd

def evaluate_monolingual_embeddings(embeddings, lower=False):
    with open('data/questions-words.txt') as f:
        lines = f.readlines()

    total_seen = defaultdict(lambda: 0)
    correct = defaultdict(lambda: 0)
    question_types = []
    not_found = 0

    for line in lines:
        if line[0] == ':':
            question_types.append(line[1:].strip())
            current_type = question_types[-1]
        else:
            total_seen[current_type] += 1
            example = line.strip().split(' ')
            if lower: example = [word.lower() for word in example]
            try:
                result = embeddings.analogy(*example[:2], example[3], 1)
                if example[2] == result[0]: correct[current_type] += 1
            except KeyError:
                not_found += 1
            
    types = []
    results = []
    for key in total_seen.keys():
        types.append(key)
        results.append(f'{correct[key]} / {total_seen[key]}')
        
    df = pd.DataFrame(data={'question type': types, 'result': results})
    display(df)
    print('Accuracy:', sum(correct.values()) / sum(total_seen.values()))
    print('Examples with missing words in the dictionary:', not_found)
    print('Total examples:', sum(total_seen.values()))

In [11]:
%%time

evaluate_monolingual_embeddings(embeddings)

,question type,result
0,capital-common-countries,397 / 506
1,capital-world,1512 / 4524
2,currency,64 / 866
3,city-in-state,463 / 2467
4,family,377 / 506
5,gram1-adjective-to-adverb,257 / 992
6,gram2-opposite,257 / 812
7,gram3-comparative,1044 / 1332
8,gram4-superlative,644 / 1122
9,gram5-present-participle,745 / 1056


Accuracy: 0.4986696684404421
Examples with missing words in the dictionary: 3175
Total examples: 19544
CPU times: user 5min 54s, sys: 3.6 s, total: 5min 57s
Wall time: 1min 29s


This is a very good result - bear in mind that we are limiting ourselves to top@1 accuracy and that we are counting synonyms as failure!

Another consideration is that while word2vec embeddings are ingenious in how efficient they are to train, they are a relatively simple way of encoding information about a language. 

Still, it is remarkable that embedding spaces posses the quality that allows us to perform operations such as the above!